In [1]:
# Import packages/modules
import numpy as np
import pandas as pd
from scipy.fft import fft, ifft
import matplotlib.pyplot as plt   
import seaborn as sns             
from matplotlib.backends.backend_pdf import PdfPages
from joblib import dump, load
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import RocCurveDisplay, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN

# Import functions
from kernelfunc import KN
from featsel import featSelect
from heatmap import heatmap
from svmtune import tuneSVM
from clustermap import clustermap

In [2]:
## Assign input variables

# classification cutoff (nM)
cutoff = 1000

# imbalance threshold (%)
imb_threshold = 35  

# number of training sets
NT = 5 

# hyper-parameter ranges
C_range = np.logspace(start= -3, stop = 3, num=7, endpoint= True, base = 10).tolist()
gamma_range = np.logspace(start= -3, stop = 3, num=7, endpoint= True, base = 10).tolist()

# data split ratio
blind_size = 0.2
val_size = 0.2

#reduction factor
red_factor = 1/3

# interval for feature selection validation
intv = 10

#sampling method (SMOTE(), ADASYN)
sampler = SMOTE()


In [3]:
# file inputs
filename_NumRep = "blosum100.csv"
filename_pepDB = "pHLA_data_20230206.csv"

In [4]:
### Notes for users
# NumRep consists of only numbers. Each row is a numeric representation of an amino acid.

In [5]:
# Import amino acids numeric representation file
NumRep = pd.read_csv(filename_NumRep, sep = ",", header = None)  
AA = ["A","R","N","D","C","Q","E","G","H","I","L","K","M","F","P","S","T","W","Y","V"]
NumRepAA = NumRep.copy()
NumRepAA['AA'] = AA

NumRepAA

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,AA
0,8,-3,-4,-5,-2,-2,-3,-1,-4,-4,...,-2,-3,-5,-2,1,-1,-6,-5,-2,A
1,-3,10,-2,-5,-8,0,-2,-6,-1,-7,...,3,-4,-6,-5,-3,-3,-7,-5,-6,R
2,-4,-2,11,1,-5,-1,-2,-2,0,-7,...,-1,-5,-7,-5,0,-1,-8,-5,-7,N
3,-5,-5,1,10,-8,-2,2,-4,-3,-8,...,-3,-8,-8,-5,-2,-4,-10,-7,-8,D
4,-2,-8,-5,-8,14,-7,-9,-7,-8,-3,...,-8,-4,-4,-8,-3,-3,-7,-6,-3,C
5,-2,0,-1,-2,-7,11,2,-5,1,-6,...,2,-2,-6,-4,-2,-3,-5,-4,-5,Q
6,-3,-2,-2,2,-9,2,10,-6,-2,-7,...,0,-5,-8,-4,-2,-3,-8,-7,-5,E
7,-1,-6,-2,-4,-7,-5,-6,9,-6,-9,...,-5,-7,-8,-6,-2,-5,-7,-8,-8,G
8,-4,-1,0,-3,-8,1,-2,-6,13,-7,...,-3,-5,-4,-5,-3,-4,-5,1,-7,H
9,-4,-7,-7,-8,-3,-6,-7,-9,-7,8,...,-6,1,-2,-7,-5,-3,-6,-4,4,I


In [6]:
# Import peptide database
pepDB = pd.read_csv(filename_pepDB, sep = ",", header = 0) 

# Assign the index numbers for all entries
pepDB['index'] = pepDB.index

# Replace *-:/ with _
trouble = ['*',"-",":","/"]
for i in trouble :
    pepDB['Allele'] = pepDB['Allele'].str.replace(i,"_") 

pepDB

,Allele,Length,Sequence,BA,index
0,HLA_DR1,14,SEFAYGSFVRTVSL,4600.0,0
1,HLA_DR15,14,SEFAYGSFVRTVSL,7100.0,1
2,HLA_DR11,14,SEFAYGSFVRTVSL,5100.0,2
3,HLA_DR13,14,SEFAYGSFVRTVSL,2000.0,3
4,HLA_DR7,14,SEFAYGSFVRTVSL,6100.0,4
...,...,...,...,...,...
63136,HLA_DRB1_09_01,15,IPFAMQMAYRFNGIG,7.9,63136
63137,HLA_DRB1_11_01,15,IPFAMQMAYRFNGIG,23.0,63137
63138,HLA_DRB1_12_01,15,IPFAMQMAYRFNGIG,2.2,63138
63139,HLA_DRB1_15_01,15,IPFAMQMAYRFNGIG,192.0,63139


In [7]:
# Get Maximum length of peptides
maxlen = pepDB['Length'].max()

# Determine signal length
power = np.ceil(np.log2(maxlen))
siglen =int(2**(power))

LenPep = int(siglen/2+1)
LenRep = NumRep.shape[1]
LenFeats =LenPep*LenRep

# Get frequency vector
freq = np.linspace(-0.5,0.5, num= siglen+1, endpoint = True)

print("Maximum Length of peptides : {}".format(maxlen))
print("Signal Length : {}".format(siglen))
print("Frequency vector : {}".format(freq))

Maximum Length of peptides : 32
Signal Length : 32
Frequency vector : [-0.5     -0.46875 -0.4375  -0.40625 -0.375   -0.34375 -0.3125  -0.28125
 -0.25    -0.21875 -0.1875  -0.15625 -0.125   -0.09375 -0.0625  -0.03125
  0.       0.03125  0.0625   0.09375  0.125    0.15625  0.1875   0.21875
  0.25     0.28125  0.3125   0.34375  0.375    0.40625  0.4375   0.46875
  0.5    ]


In [8]:
# Encode peptide sequences using Fast Fourier Transform 

c = 0
fTab = np.zeros((len(pepDB['Sequence']),int((siglen/2+1)*LenRep)))   

for pept in pepDB['Sequence'] :
    p = list(pept)  
    nTab = np.zeros((len(p), LenRep)) 
    
    for i in range(0,len(p)) :
        if p[i] in AA :    
            nTabr = NumRepAA[NumRepAA['AA'].str.contains(p[i],case=False)].drop('AA', axis =1).to_numpy() 
            nTab[i,:] = nTabr  
         
    feats = []  
    N = len(pept) 
    
    for nn in range(0,LenRep) :
        tmp = [0]*(siglen)
        
        for j in range(0,N) :
            tmp[j] = nTab[j,nn]  
    
        # fast fourier transform
        FT = fft(tmp)  
        FT = np.append(FT,FT[0])  
    
        feats_tmp = []
        
        for k in range(int(siglen/2),int(siglen+1)) :  
            feats_tmp = np.append(feats_tmp, abs(FT[k])/N)  
                                                            
        feats = np.append(feats, feats_tmp)
    
    fTab[c,:] = feats  
    c += 1   




fTab = pd.DataFrame(fTab)
fTab.to_csv('fTab.csv', index = False)

fTab
#plt.plot(freq, FT)
#plt.plot(freq, abs(FT))

,0,1,2,3,4,5,6,7,8,9,...,330,331,332,333,334,335,336,337,338,339
0,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,1.173286,1.585972,1.363775,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429
1,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,1.173286,1.585972,1.363775,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429
2,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,1.173286,1.585972,1.363775,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429
3,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,1.173286,1.585972,1.363775,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429
4,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,1.173286,1.585972,1.363775,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63136,1.866667,1.832913,1.361859,0.433224,0.625032,0.873569,0.783019,1.048860,1.534782,1.460086,...,1.120606,0.359011,0.649737,0.242819,0.523328,0.767363,0.832507,0.932943,2.406720,3.133333
63137,1.866667,1.832913,1.361859,0.433224,0.625032,0.873569,0.783019,1.048860,1.534782,1.460086,...,1.120606,0.359011,0.649737,0.242819,0.523328,0.767363,0.832507,0.932943,2.406720,3.133333
63138,1.866667,1.832913,1.361859,0.433224,0.625032,0.873569,0.783019,1.048860,1.534782,1.460086,...,1.120606,0.359011,0.649737,0.242819,0.523328,0.767363,0.832507,0.932943,2.406720,3.133333
63139,1.866667,1.832913,1.361859,0.433224,0.625032,0.873569,0.783019,1.048860,1.534782,1.460086,...,1.120606,0.359011,0.649737,0.242819,0.523328,0.767363,0.832507,0.932943,2.406720,3.133333


In [9]:
fTab = pd.read_csv("fTab.csv", sep    = ",", header = 0)

# Assign lables based on a cutoff (binary classification)
trainInput = pd.DataFrame(fTab)
Class = []

for i in range(0,len(pepDB)) :
    if pepDB.loc[i].at["BA"] > cutoff :
        Class.append('NB')
    else :
        Class.append('B')

# get trainInput for training and pepDB for statistics
trainInput['Class'] = Class
trainInput['index'] = pepDB['index']
trainInput['Allele'] = pepDB['Allele']
pepDB['Class'] = trainInput['Class']

pepDB

,Allele,Length,Sequence,BA,index,Class
0,HLA_DR1,14,SEFAYGSFVRTVSL,4600.0,0,NB
1,HLA_DR15,14,SEFAYGSFVRTVSL,7100.0,1,NB
2,HLA_DR11,14,SEFAYGSFVRTVSL,5100.0,2,NB
3,HLA_DR13,14,SEFAYGSFVRTVSL,2000.0,3,NB
4,HLA_DR7,14,SEFAYGSFVRTVSL,6100.0,4,NB
...,...,...,...,...,...,...
63136,HLA_DRB1_09_01,15,IPFAMQMAYRFNGIG,7.9,63136,B
63137,HLA_DRB1_11_01,15,IPFAMQMAYRFNGIG,23.0,63137,B
63138,HLA_DRB1_12_01,15,IPFAMQMAYRFNGIG,2.2,63138,B
63139,HLA_DRB1_15_01,15,IPFAMQMAYRFNGIG,192.0,63139,B


In [10]:
trainInput

,0,1,2,3,4,5,6,7,8,9,...,333,334,335,336,337,338,339,Class,index,Allele
0,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429,NB,0,HLA_DR1
1,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429,NB,1,HLA_DR15
2,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429,NB,2,HLA_DR11
3,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429,NB,3,HLA_DR13
4,0.000000,1.030808,1.416511,1.078329,1.297381,1.708193,1.246408,0.755927,0.857143,0.369575,...,0.728250,0.535064,0.442527,1.082060,1.706949,1.992956,2.071429,NB,4,HLA_DR7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63136,1.866667,1.832913,1.361859,0.433224,0.625032,0.873569,0.783019,1.048860,1.534782,1.460086,...,0.242819,0.523328,0.767363,0.832507,0.932943,2.406720,3.133333,B,63136,HLA_DRB1_09_01
63137,1.866667,1.832913,1.361859,0.433224,0.625032,0.873569,0.783019,1.048860,1.534782,1.460086,...,0.242819,0.523328,0.767363,0.832507,0.932943,2.406720,3.133333,B,63137,HLA_DRB1_11_01
63138,1.866667,1.832913,1.361859,0.433224,0.625032,0.873569,0.783019,1.048860,1.534782,1.460086,...,0.242819,0.523328,0.767363,0.832507,0.932943,2.406720,3.133333,B,63138,HLA_DRB1_12_01
63139,1.866667,1.832913,1.361859,0.433224,0.625032,0.873569,0.783019,1.048860,1.534782,1.460086,...,0.242819,0.523328,0.767363,0.832507,0.932943,2.406720,3.133333,B,63139,HLA_DRB1_15_01


In [11]:
# Scale input X


X = trainInput.iloc[:,0:LenFeats]

scaler = StandardScaler()
scaler.fit(X)
X_scaled_all = scaler.transform(X)

trainInputScaled = pd.concat([ pd.DataFrame(X_scaled_all), trainInput['Class'], trainInput['index'],trainInput['Allele']], axis=1)

trainInputScaled

,0,1,2,3,4,5,6,7,8,9,...,333,334,335,336,337,338,339,Class,index,Allele
0,-1.233000,0.684613,1.466916,0.724206,1.239944,2.169633,1.066395,-0.035739,0.148650,-0.851859,...,-0.490349,-0.925757,-1.049516,-0.102605,1.412809,0.085375,-0.640885,NB,0,HLA_DR1
1,-1.233000,0.684613,1.466916,0.724206,1.239944,2.169633,1.066395,-0.035739,0.148650,-0.851859,...,-0.490349,-0.925757,-1.049516,-0.102605,1.412809,0.085375,-0.640885,NB,1,HLA_DR15
2,-1.233000,0.684613,1.466916,0.724206,1.239944,2.169633,1.066395,-0.035739,0.148650,-0.851859,...,-0.490349,-0.925757,-1.049516,-0.102605,1.412809,0.085375,-0.640885,NB,2,HLA_DR11
3,-1.233000,0.684613,1.466916,0.724206,1.239944,2.169633,1.066395,-0.035739,0.148650,-0.851859,...,-0.490349,-0.925757,-1.049516,-0.102605,1.412809,0.085375,-0.640885,NB,3,HLA_DR13
4,-1.233000,0.684613,1.466916,0.724206,1.239944,2.169633,1.066395,-0.035739,0.148650,-0.851859,...,-0.490349,-0.925757,-1.049516,-0.102605,1.412809,0.085375,-0.640885,NB,4,HLA_DR7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63136,2.253105,2.506928,1.344711,-0.701839,-0.271509,0.270761,0.043332,0.572171,1.508875,1.422377,...,-1.405822,-0.946644,-0.464209,-0.531952,-0.087819,0.682686,0.383551,B,63136,HLA_DRB1_09_01
63137,2.253105,2.506928,1.344711,-0.701839,-0.271509,0.270761,0.043332,0.572171,1.508875,1.422377,...,-1.405822,-0.946644,-0.464209,-0.531952,-0.087819,0.682686,0.383551,B,63137,HLA_DRB1_11_01
63138,2.253105,2.506928,1.344711,-0.701839,-0.271509,0.270761,0.043332,0.572171,1.508875,1.422377,...,-1.405822,-0.946644,-0.464209,-0.531952,-0.087819,0.682686,0.383551,B,63138,HLA_DRB1_12_01
63139,2.253105,2.506928,1.344711,-0.701839,-0.271509,0.270761,0.043332,0.572171,1.508875,1.422377,...,-1.405822,-0.946644,-0.464209,-0.531952,-0.087819,0.682686,0.383551,B,63139,HLA_DRB1_15_01


In [12]:
# Get dataframes by alleles

Allele_list = trainInputScaled['Allele'].drop_duplicates().tolist()
NumAllele = len(Allele_list)
AS_tIS = {}
for Alleles in Allele_list :
    AS_tIS[Alleles]=trainInputScaled[trainInputScaled['Allele'].str.match(Alleles)]
    
Allele_list

['HLA_DR1',
 'HLA_DR15',
 'HLA_DR11',
 'HLA_DR13',
 'HLA_DR7',
 'HLA_DRB1_15_01',
 'HLA_DRB1_01_01',
 'HLA_DRB1_04_01',
 'HLA_DRB1_07_01',
 'HLA_DRB1_08_02',
 'HLA_DRB1_09_01',
 'HLA_DRB1_12_01',
 'HLA_DRB5_01_01',
 'HLA_DRB1_04_05',
 'HLA_DRB1_13_02',
 'HLA_DRB1_03_01',
 'HLA_DRB1_11_01',
 'HLA_DRB4_01_01',
 'HLA_DRB3_01_01',
 'HLA_DRB1_08_01',
 'HLA_DRA_01_01_DRB1_04_01',
 'HLA_DRA_01_01_DRB1_04_04',
 'HLA_DQA1_03_01_DQB1_03_01',
 'HLA_DR4',
 'HLA_DR3',
 'HLA_DRB1_04_04',
 'HLA_DQA1_03_01_DQB1_03_02',
 'HLA_DQA1_05_01_DQB1_02_01',
 'HLA_DRB1_04_02',
 'HLA_DRA_01_01_DRB1_08_01',
 'HLA_DRB1_13_01',
 'HLA_DRB1_16_02',
 'HLA_DR2',
 'HLA_DQA1_05_01_DQB1_03_01',
 'HLA_DPB1_04_01',
 'HLA_DPB1_04_02',
 'HLA_DQB1_06_02',
 'HLA_DQA1_01_02_DQB1_06_02',
 'HLA_DQ8',
 'HLA_DPA1_01_03_DPB1_02_01',
 'HLA_DPA1_01_03_DPB1_04_01',
 'HLA_DPB1_05_01',
 'HLA_DQB1_03_01',
 'HLA_DQB1_05_01',
 'HLA_DQB1_03_02',
 'HLA_DQB1_02_01',
 'HLA_DRA_01_01_DRB1_01_01',
 'HLA_DRB1_10_01',
 'HLA_DRB1_04_03',
 'HLA_DQA1_0

In [13]:
AS_tIS['HLA_DRB1_04_01']

,0,1,2,3,4,5,6,7,8,9,...,333,334,335,336,337,338,339,Class,index,Allele
8,-1.046245,1.559588,0.358044,-0.295584,0.116935,0.802836,0.035925,0.226857,1.079719,0.440761,...,0.764082,1.096730,-1.249960,0.025598,-0.165324,-0.883590,0.061979,B,8,HLA_DRB1_04_01
16,0.634556,-0.539684,-0.834291,-0.492346,-0.834410,-0.262717,0.155923,0.061941,-1.020139,0.359025,...,0.543649,0.829170,-1.373656,0.470828,0.169278,-1.367893,-0.645479,B,16,HLA_DRB1_04_01
26,-0.112467,-0.755847,-0.602791,-0.685685,-1.402568,-0.538711,0.151080,-0.002773,-0.902793,-1.302961,...,0.349431,0.373065,-0.031481,-1.113927,0.255278,-1.494803,-2.060394,B,26,HLA_DRB1_04_01
37,-1.108497,-1.259115,-1.229961,-1.307537,-0.500081,-0.938517,-0.996372,-0.334363,-0.757898,-1.383804,...,1.325195,0.400997,0.954134,-0.092135,0.754320,-0.117331,-0.774107,NB,37,HLA_DRB1_04_01
46,-0.112467,-0.520431,-0.988412,-0.760171,-0.495690,-0.229229,0.368238,0.328249,0.039513,0.577853,...,-0.792456,0.580279,-0.280665,-0.570771,-0.923230,0.365107,0.705123,NB,46,HLA_DRB1_04_01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63115,-1.233000,-0.746611,-0.472823,-0.940403,-1.263020,-1.377015,-0.580467,-0.712180,0.045060,0.174085,...,-0.497221,-0.430704,-1.705269,-0.840634,-0.984214,1.176621,1.219638,B,63115,HLA_DRB1_04_01
63123,-0.859489,-1.283458,-1.255244,-0.750162,-0.275295,-0.060863,-0.132903,-0.826887,-1.170433,-1.158391,...,-0.174886,-0.498276,-1.182041,0.616538,-0.607404,0.539618,1.026694,B,63123,HLA_DRB1_04_01
63128,-0.610482,-0.581707,1.633334,0.249365,0.177381,0.141093,0.273730,1.797311,-1.089398,1.407188,...,1.128303,0.064825,-1.017580,-0.923881,0.321062,1.401712,1.026694,B,63128,HLA_DRB1_04_01
63133,2.253105,2.506928,1.344711,-0.701839,-0.271509,0.270761,0.043332,0.572171,1.508875,1.422377,...,-1.405822,-0.946644,-0.464209,-0.531952,-0.087819,0.682686,0.383551,B,63133,HLA_DRB1_04_01


In [14]:
# Create dataframes for statistics

CL0 = ['Alleles', 'N_total', 'N_B', 'N_NB', 'P_B', 'P_NB']
DataStats = pd.DataFrame(columns = CL0)

CL1 = ['Allele'] + list(range(1,LenFeats+1))
FeatRanks = pd.DataFrame(columns = CL1)

numFeat = []
for i in range(intv,LenFeats+1,intv) :
    numFeat.append(i)
CL2 = ['Allele'] + numFeat
FeatSetAccu = pd.DataFrame(columns = CL2)

CL3 = ['Allele', 'Number of features used']
NumFeatUsed = pd.DataFrame(columns = CL3)

CL4 = ['Allele', 'best C', 'best gamma', 'Max AUC' ]
bp_max_auc = pd.DataFrame(columns = CL4)

CL5 = ['Allele', 'Train AUC', 'Prediction AUC']
TrainAUC_BlindAUC = pd.DataFrame(columns = CL5)

In [ ]:
### SVM

X_Blind_Sets = {}
y_Blind_Sets = {}
AS_cgauc = {}

heatmapGS = PdfPages('Heatmaps_GridSearch.pdf')
featselplot = PdfPages('FeatureSelectionPlot.pdf')
ROCcurve_AUC = PdfPages('ROCcurve_AUC')

for Alleles in Allele_list :
    
    ## data preparation for training and cross-validation
    
    # get allele-specific dataset
    dataset_SVM = AS_tIS[Alleles] 
    X_scaled = dataset_SVM.iloc[:,0:LenFeats]
    y = dataset_SVM['Class']
    
    # save data statistics
    N_total = len(dataset_SVM)
    N_B = dataset_SVM['Class'].str.match('B').sum()
    N_NB = dataset_SVM['Class'].str.match('NB').sum()
    P_B = N_B/N_total*100
    P_NB = N_NB/N_total*100

    DataStats.loc[len(DataStats.index)] = [Alleles, N_total, N_B, N_NB, P_B, P_NB]
    
    # OverSampling/SMOTE for imbalanced data
    if P_B < imb_threshold or P_NB < imb_threshold :
        X_scaled, y = sampler.fit_resample(X_scaled, y)
        
    # split dataset into train set and blind set
    X_train_all, X_blind, y_train_all, y_blind = train_test_split(X_scaled, y,stratify =y, random_state= None, test_size= blind_size)

    X_Blind_Sets[Alleles] = X_blind 
    y_Blind_Sets[Alleles] = y_blind
    
    # split train set into train and test(val) data / repeat NT times and |get NT number of datsets
    N_test = int(np.ceil(len(X_train_all)*0.2))
    X_train_sets = np.empty((NT,len(X_train_all)-N_test,LenFeats))
    X_test_sets = np.empty((NT,N_test,LenFeats))      
    y_train_sets =  [[] for i in range(NT)]   
    y_test_sets =[[] for i in range(NT)]   

    for i in range(0,NT) :
        X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, stratify = y_train_all, random_state= None, test_size= val_size)
        X_train_sets[i,:,:] = X_train
        X_test_sets[i,:,:]  = X_test
        y_train_sets[i] = y_train
        y_test_sets[i] = y_test


    ## feature selection

    # generate feature rank using function featSelect
    featRank = featSelect(X_train_all, y_train_all, X_train_sets, y_train_sets, X_test_sets, y_test_sets, LenFeats, C_range, gamma_range, redF = red_factor)
    
    sf =  np.argsort(featRank).mean(axis=0).argsort() 
    
    allelesf = [Alleles] + sf.tolist()
    FeatRanks.loc[len(FeatRanks.index)] = allelesf

    # Validation of feature selection
    featAccu = []
    nFeat = []

    for i in range(intv, LenFeats +1,intv) :      
        selFeat = sf[0:i]
        tunePer = tuneSVM(X_train_all, y_train_all, X_train_sets[:,:,selFeat], y_train_sets, X_test_sets[:,:,selFeat], y_test_sets, costs= C_range, gammas = gamma_range)
        nFeat.append(i)
        featAccu.append(tunePer[:,2].max())
    
    AlleFeatAccu = [Alleles] + featAccu
    FeatSetAccu.loc[len(FeatSetAccu.index)] = AlleFeatAccu
    
    # feature selection plot
    fig, ax = plt.subplots(1,1, figsize=(7, 4))
    plt.plot(nFeat, featAccu) 
    plt.ylabel('Validation Accuracy')
    plt.xlabel('Number of features')
    plt.title('Validation Accuracy with sets of features  '+'['+Alleles+']')
    plt.show()

    featselplot.savefig(fig)
    ## train SVM model with the selected features
    
    # Prepare data with the selected features
    NumFeat = nFeat[featAccu.index(max(featAccu))]  
    feats_id = sf[0:NumFeat] 

    NumFeatUsed.loc[len(NumFeatUsed.index)] = NumFeat
    
    X_train_all = X_train_all.to_numpy()
    X_blind = X_blind.to_numpy()

    X_train_all_sf = X_train_all[:, feats_id]
    y_train_all_sf = y_train_all

    X_train_sets_sf = X_train_sets[:,:, feats_id]
    y_train_sets_sf = y_train_sets
    X_test_sets_sf = X_test_sets[:,:, feats_id]
    y_test_sets_sf = y_test_sets

    X_blind_sf = X_blind[:, feats_id]
    y_blind_sf = y_blind

    # grid search
    tunePer_sf = tuneSVM(X_train_all_sf, y_train_all_sf, X_train_sets_sf, y_train_sets_sf, X_test_sets_sf, y_test_sets_sf, costs= C_range, gammas = gamma_range)
    AS_cgauc[Alleles]= tunePer_sf  # save cgaccu
    
    # heatmaps
    aucs = np.zeros(((len(C_range), len(gamma_range))))
    for i in range(0,len(C_range)) :
        aucs[i,:] = tunePer_sf[i*7:(i+1)*7,2]
    heatmap(heatmapGS, aucs, Alleles, C_range, gamma_range)
    

    # best parameters from grid search        
    max_auc_id = tunePer_sf[:,2].argmax(axis = 0)
    bp_c = tunePer_sf[max_auc_id, 0]
    bp_g = tunePer_sf[max_auc_id, 1]
    max_auc = tunePer_sf[max_auc_id,2]

    bp_max_auc.loc[len(bp_max_auc.index)] = [Alleles, bp_c, bp_g, max_auc]   # save best params and max accuracy


    # fit SVM
    svc = SVC(kernel = 'rbf', C =bp_c, gamma =bp_g).fit(X_train_all_sf, y_train_all_sf)
    dump(svc, 'SVM_{}.joblib'.format(Alleles)) # save SVMs / Use when reload [clf = load('SVM_{}.joblib'.format(Alleles))] 
    
    print("MHC Allele : {}".format(Alleles))
    
    auc_train = roc_auc_score(y_train_all_sf, svc.decision_function(X_train_all_sf))
    print("Training AUC : {}".format(auc_train))

    # validate the SVM with blind test set
    auc_blind = roc_auc_score(y_blind_sf, svc.decision_function(X_blind_sf))
    print("Blind Test AUC : {}".format(auc_blind))
    
    # ROC Curve
    svc_disp = RocCurveDisplay.from_estimator(svc, X_blind_sf, y_blind_sf, pos_label ='B')
    ROCcurve = svc_disp
    plt.show()
    #ROCcurve_AUC.savefig(ROCcurve)
    
    TrainAUC_BlindAUC.loc[len(TrainAUC_BlindAUC.index)] = [Alleles, auc_train, auc_blind]

heatmapGS.close()
featselplot.close()
#ROCcurve_AUC.close()

# save results
DataStats.to_csv('DataStatistics.csv', index = False)
FeatRanks.to_csv('FeatureRanks.csv', index = False) # from 0 to 169
FeatSetAccu.to_csv('FeatureSetsAccuracy.csv', index = False)
NumFeatUsed.to_csv('NumberOfFeaturesUsed.csv',index = False)
#AS_cgaccu.to_csv('C_gamma_Accuracy_TuneSVM.csv', index = False)  # dict to csv how ????
bp_max_auc.to_csv('BestParams(C,g)_MaxAccuracy_TuneSVM.csv', index = False) # merge these two
TrainAUC_BlindAUC.to_csv('Training_BlindTest_AUC.csv', index = False)  # merge these two

In [ ]:
DataStats.sort_values(by = 'P_NB')

In [ ]:
FeatRanks

In [ ]:
# ClusterMap

FeatRanks_heatmap = pd.DataFrame(FeatRanks)
FeatRanks_heatmap = FeatRanks.drop('Allele', axis=1)


for i in range(len(Allele_list)):
    FeatRanks_heatmap.iloc[i,:] = np.argsort(FeatRanks_heatmap.iloc[i,:])
    FeatRanks_heatmap.iloc[i,:] = FeatRanks_heatmap.iloc[i,:].rank(ascending = False)# the higher the number, the more important the feature
FeatRanks_heatmap.columns = range(FeatRanks_heatmap.columns.size) # reset column index # x-axis : feature  y-axis : allele
FeatRanks_heatmap

In [ ]:
FreqComp = pd.DataFrame(0, index = range(NumAllele), columns = range(LenPep))

for i in range(LenPep):
    for j in range(LenRep) :
        FreqComp[i] = FreqComp[i] + FeatRanks_heatmap[LenPep*j+i]

FreqComp

In [ ]:
clustermap(FreqComp, FeatRanks, LenPep, 'Alleles', 'Frequency Component')

In [ ]:
NumRepComp = pd.DataFrame(0, index = range(NumAllele), columns = range(LenRep))

for i in range(LenRep):
    for j in range(LenPep) :
        NumRepComp[i] = NumRepComp[i] + FeatRanks_heatmap[LenPep*i+j]

NumRepComp

In [ ]:
clustermap(NumRepComp, FeatRanks, LenRep, 'Alleles', 'Numeric Representation Component')

In [ ]:
Total_FreqComp_Sum = []
for j in range(LenPep):
    Total_FreqComp_Sum.append(FreqComp[j].sum()/LenRep)

Total_NumRepComp_Sum = []
for i in range(LenRep):
    Total_NumRepComp_Sum.append(NumRepComp[i].sum()/LenPep)

Total_Features_Sum = []
for k in range(LenFeats):
    Total_Features_Sum.append(FeatRanks_heatmap[k].sum())


In [ ]:
plt.plot(range(LenRep), Total_NumRepComp_Sum)
print(sum(Total_NumRepComp_Sum)*LenPep)

In [ ]:
plt.plot(range(LenPep), Total_FreqComp_Sum)
print(sum(Total_FreqComp_Sum)*LenRep)

In [ ]:
plt.plot(range(LenFeats), Total_Features_Sum)
print(sum(Total_Features_Sum))

In [ ]:
FeatSetAccu

In [ ]:
NumFeatUsed

In [ ]:
NAllele = [] # Allele Frequency
NFeatUniq = NumFeatUsed['Number of features used'].unique().tolist()
NFeatUniq_S = sorted(NFeatUniq)  # sort() returns None, sorted() returns a sorted list


for NFeat in NFeatUniq_S :
    NAllele.append((NumFeatUsed['Number of features used'].loc[NumFeatUsed['Number of features used']==NFeat].count())*100/44)

bp = sns.barplot(x = NFeatUniq_S, y = NAllele)  # NFeat vs Percentage of NALlele
bp.set_yticks(range(0,50,5))
plt.show()


In [ ]:
AS_cgaccu

In [ ]:
ValAccu_BlindAccu

In [ ]:
#PredAccu_AVG 
Sample_Size = []
#ValAccu_BlindAccu['Sample Size'] = np.zeros(len(ValAccu_BlindAccu))
for Alleles in Allele_list :
    Sample_Size.append(len(AS_tIS[Alleles]))
ValAccu_BlindAccu['Sample Size'] = Sample_Size

PASS = []
for i in range(len(ValAccu_BlindAccu)):
    PASS.append((ValAccu_BlindAccu.iloc[i,2])*(ValAccu_BlindAccu.iloc[i,3]))
ValAccu_BlindAccu['PA X SS'] = PASS

OverallPredictionAccuracy = sum(PASS)/sum(Sample_Size)
print('Overall Blind Test Accuracy : {}'.format(OverallPredictionAccuracy))


plt.figure(figsize=(15,8))
AllAccu = sns.barplot(x = ValAccu_BlindAccu['Allele'], y = ValAccu_BlindAccu['Prediction Accuracy'])  # NFeat vs Percentage of NALlele

AllAccu.set_xticklabels(AllAccu.get_xticklabels(), rotation = 45, horizontalalignment = 'right')
plt.show()


In [ ]:
FeatSetAccu
for i in range(NumAllele) :
    plt.plot(numFeat, FeatSetAccu.iloc[i,1:int(LenFeats/intv)+1])

In [ ]:
# for ROC
#clf = load('SVMHLA_DPA1_01_03_DPB1_02_01.joblib')
#clf.score(X_train_all_sf, y_train_all_sf)

In [ ]:
DataStats

In [ ]:
DataStats['P_B'] 

In [ ]:
ValAccu_BlindAccu['P_NB'] = DataStats['P_NB'] 
ValAccu_BlindAccu

In [ ]:

sns.relplot(data=ValAccu_BlindAccu, x="Sample Size", y="Prediction Accuracy", hue ='P_NB', height = 5, aspect = 10/5)


In [ ]:

zoomin = sns.relplot(data=ValAccu_BlindAccu, x="Sample Size", y="Prediction Accuracy", hue ='P_NB', height = 5, aspect = 10/5)

zoomin.ax.set_xlim(0,1200)
plt.show()

In [ ]:

sns.relplot(data=ValAccu_BlindAccu, x="Sample Size", y="Validation Accuracy", hue ='P_NB',height = 5, aspect = 10/5)

In [ ]:
ValAccu_BlindAccu.sort_values(by = 'Sample Size', ascending = False)